In [148]:
import os
import json
import requests

import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt


In [149]:
# Replace this with the path to your pickled file
pickle_file_path = 'scraping\gme_daily_transformed_df.pkl'

# Load the DataFrame from the pickle file
gme_daily_transformed_df = pd.read_pickle(pickle_file_path)

In [150]:
# Filtering the DataFrame to include only data from December 2020 to April 2021 and creating a copy
gme_jan_apr2021_df = gme_daily_transformed_df[(gme_daily_transformed_df['Date'] >= '2021-01-01') & (gme_daily_transformed_df['Date'] <= '2021-04-30')].copy()

In [151]:
gme_jan_apr2021_df['Event'] = pd.NA
gme_jan_apr2021_df.head()


Date    Open      High       Low   Close      Volume Event
647 2021-04-30  175.00  183.8000  172.7000  173.59   4433565.0  <NA>
648 2021-04-29  176.56  180.8600  170.7500  176.19   3760250.0  <NA>
649 2021-04-28  172.10  181.9800  166.0281  178.58   5788938.0  <NA>
650 2021-04-27  183.55  188.0000  173.1700  177.77   8932718.0  <NA>
651 2021-04-26  150.98  174.6795  148.7600  168.93  10532108.0  <NA>

In [152]:
# Reset the index to make 'Date' a regular column
gme_jan_apr2021_df.reset_index(inplace=True)

# Assuming 'gme_jan_apr2021_df' is your DataFrame

# Convert 'Date' to datetime if it's not already
gme_jan_apr2021_df['Date'] = pd.to_datetime(gme_jan_apr2021_df['Date'])

# Use the 'Date' column to assign events
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-02-19'), 'Event'] = 'February'
gme_jan_apr2021_df.loc[gme_jan_apr2021_df['Date'] == pd.to_datetime('2021-01-07'), 'Event'] = 'January'

# Keep 'Date' as a datetime object for further operations


# If you want to set 'Date' back as the index
# gme_jan_apr2021_df.set_index('Date', inplace=True)

In [153]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices with hover text
fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Close'],
        mode='lines+markers',
        name='Stock Price',
        hoverinfo='none'  # Display hover text
        #text=df['Event']  # Set hover text to be the 'Event' column
    )
)

# Customize hover text for each point
for index, row in df.iterrows():
    if pd.notnull(row['Event']):  # Check if there is an event
        fig.add_trace(
            go.Scatter(
                x=[row['Date']],
                y=[row['Close']],
                text=["<b>{}</b>".format(row['Event'])],  # Bold the event text
                mode='markers+text',
                hoverinfo='text',
                showlegend=False,
                marker=dict(
                    size=12,
                    color='red'),
                hoverlabel=dict(
                    bgcolor='white',  # Background color of the hover label
                    font_size=16,  # Font size of the text
                    font_family='Rockwell'  # Font of the text
                )
            )
        )

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    hovermode='closest'  # Show the hover info for the closest point
)

# Show the plot in the notebook if needed
# fig.show()

# Save the plot as HTML
fig.write_html("interactive_timeline.html")


In [154]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Stock_Price', 'Event' columns
df = gme_jan_apr2021_df

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines+markers', name='Stock Price'))

# Add annotations for each event
for index, row in df.iterrows():
    if pd.notnull(row['Event']):  # Assuming NaNs where there's no event
        fig.add_annotation(
            x=row['Date'], 
            y=row['Close'], 
            text=row['Event'], 
            showarrow=True,
            arrowhead=1
        )

# Add shapes if necessary
# fig.add_shape(...)

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(rangeslider=dict(visible=True), type='date')
)

# Show the plot
#fig.show()

# Save the plot as HTML
fig.write_html("interactive_timeline.html")

# Or deploy as a web app using Dash (requires additional code)


In [155]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines+markers', name='Stock Price'))

# Add annotations for each event
for index, row in df.iterrows():
    if pd.notnull(row['Event']):  # Assuming NaNs where there's no event
        # Add an annotation without custom date formatting
        fig.add_annotation(
            x=row['Date'], 
            y=row['Close'], 
            text=row['Event'],  # Directly use the event text
            showarrow=True,
            arrowhead=1,
            arrowcolor="red",
            arrowsize=3,
            ax=0,
            ay=-40,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=0.8
        )
        # Add a circle shape to highlight the event
        fig.add_shape(
            # Circle parameters have been slightly adjusted for better visibility
            type="circle",
            xref="x",
            yref="y",
            fillcolor="red",
            x0=pd.to_datetime(row['Date']) - pd.Timedelta(days=1),  # Start a day before the event
            y0=row['Close'] - 10,  # 10 units less than the event price
            x1=pd.to_datetime(row['Date']) + pd.Timedelta(days=1),  # End a day after the event
            y1=row['Close'] + 10,  # 10 units more than the event price
            line_color="red",
        )

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    showlegend=False
)

# Show the plot in the notebook if needed
# fig.show()

# Save the plot as HTML
fig.write_html("interactive_timeline.html")


# Plot with red markers

In [156]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue')  # Default size and color of markers
))

# Add invisible markers for each event that will display the event text on hover
for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[row['Event']],
            mode='markers',
            marker=dict(size=10, color='red', opacity=1),  # Red marker
            hoverinfo='text+name',
            name=row['Event'],
            showlegend=False
        ))

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    )
)

fig.write_html("interactive_timeline.html")

In [158]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue')  # Default size and color of markers
))

# Add invisible markers for each event that will display the event text on hover
for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        # Add a red marker
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[row['Event']],
            mode='markers',
            marker=dict(size=10, color='red', opacity=1),  # Red marker
            hoverinfo='text+name',
            name=row['Event'],
            showlegend=False
        ))
        # Add a triangle pointing upwards to the event marker
        fig.add_shape(
            type="path",
            path=f"M{row['Date']},{row['Close'] - 10} L{pd.to_datetime(row['Date']) - pd.Timedelta(days=0.2)},{row['Close'] - 30} L{pd.to_datetime(row['Date']) + pd.Timedelta(days=0.2)},{row['Close'] - 30} Z",
            fillcolor="red",
            line_color="red",
        )

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    )
)

# Show the plot
fig.write_html("interactive_timeline.html")


In [159]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue')  # Default size and color of markers
))

# Add invisible markers for each event that will display the event text on hover
for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        # Add a red marker
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[row['Event']],
            mode='markers',
            marker=dict(size=12, color='red'),  # Red marker
            hoverinfo='text',
            showlegend=False
        ))
        # Add a triangle by creating a scatter plot with custom markers
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[""],  # no text
            mode='markers',
            marker=dict(
                symbol='triangle-up',  # Triangle symbol
                size=16,  # Size of the triangle marker
                color='red'  # Color of the triangle marker
            ),
            hoverinfo='skip',  # No hover info for the triangle
            showlegend=False
        ))

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    ),
    hovermode='closest'
)

# Show the plot
fig.write_html("interactive_timeline.html")


In [161]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue')  # Default size and color of markers
))

# Add invisible markers for each event that will display the event text on hover
for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        # Add a red marker
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[row['Event']],
            mode='markers',
            marker=dict(size=12, color='red'),  # Red marker
            hoverinfo='text',
            showlegend=False
        ))
        # Add a triangle by creating a scatter plot with custom markers
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[""],  # no text
            mode='markers',
            marker=dict(
                symbol='triangle-down',  # Triangle symbol
                size=16,  # Size of the triangle marker
                color='red'  # Color of the triangle marker
            ),
            hoverinfo='skip',  # No hover info for the triangle
            showlegend=False
        ))

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    ),
    hovermode='closest'
)

# Show the plot
fig.write_html("interactive_timeline.html")


# plot with red triangles

In [165]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is a DataFrame with 'Date', 'Close', and 'Event' columns
df = gme_jan_apr2021_df

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Create the base timeline chart
fig = go.Figure()

# Add line for stock prices
fig.add_trace(go.Scatter(
    x=df['Date'], 
    y=df['Close'], 
    mode='lines+markers',
    name='Stock Price',
    marker=dict(size=6, color='blue')  # Default size and color of markers
))

# Determine an appropriate offset for the triangle marker based on your data's scale
y_offset = (max(df['Close']) - min(df['Close'])) * 0.05  # Example: 2% of the data range

# Add invisible markers for each event that will display the event text on hover
for index, row in df.iterrows():
    if pd.notnull(row['Event']):
        # Add a red marker
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close']],
            text=[row['Event']],
            mode='markers',
            marker=dict(size=12, color='red'),  # Red marker
            hoverinfo='text',
            showlegend=False
        ))
        # Add a triangle by creating a scatter plot with custom markers
        fig.add_trace(go.Scatter(
            x=[row['Date']],
            y=[row['Close'] + y_offset],  # Add the offset to the y-coordinate
            text=[""],  # no text
            mode='markers',
            marker=dict(
                symbol='triangle-down',  # Triangle symbol
                size=16,  # Size of the triangle marker
                color='red'  # Color of the triangle marker
            ),
            hoverinfo='skip',  # No hover info for the triangle
            showlegend=False
        ))

# Customize the layout
fig.update_layout(
    title='GameStop Stock Surge Interactive Timeline',
    xaxis_title='Date',
    yaxis_title='Stock Price ($)',
    xaxis=dict(
        rangeslider=dict(visible=True), 
        type='date'
    ),
    hovermode='closest'
)

# Show the plot
fig.write_html("interactive_timeline.html")
